In [289]:
from bs4 import BeautifulSoup
import requests 
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from datetime import datetime



def find_crypto_information(soup):
    
    data = []    
    date_str = soup.find('h1').text.strip('\n').split('-')
    date = datetime.strptime(date_str[1], ' %B %d, %Y')
    table = soup.find('table', id='currencies-all')
    rows = table.find_all('tbody')   
    for row in rows:
        try:
            symbol = row.find('td', class_='text-left col-symbol').text
            price = row.find('a', class_='price').text
        except AttributeError:
            print('error')
            continue
            
        data.append([symbol, price,date])
        
    return data

#Only stores historical links
def get_urls(soup):
    urls = []
    url_base = 'https://coinmarketcap.com' 
    for link in soup.find_all('a',href=True):
        link_text = link['href']
        if fuzz.partial_ratio('/historical/201',link_text) == 100 and \
            fuzz.partial_ratio('/201',link_text) == 100 and link_text!='/':
            urls.append(url_base + link_text)
    return urls

def downsample_dates(urls,range1,range2):
    new_list = []
    for url in urls:
        if fuzz.partial_ratio(url,range1) != 100 | \
           fuzz.partial_ratio(url,range2) != 100 :
            continue
        new_list.append(url)
        return new_list
        

r = requests.get('https://coinmarketcap.com/historical/')
soup = BeautifulSoup(r.text, 'html5lib')    
urls = get_urls(soup)
range1 = '20180708'
range2 = '20170709'
downsample_dates(urls,range1,range2)

['https://coinmarketcap.com/historical/20170709/']

In [290]:
#Now get the historical prices

In [ ]:

price_data = []
for url in urls:
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html5lib')
    output = find_crypto_information(soup)
    print(output)
        
        
    